##### Author: Jimin Kim (jk55@uw.edu)
##### Version 1.5.0

# Lab 4 Report

## Group Members: Tuan Huynh, Caitlin DeShazo-Couchot
## Group Name for Leaderboard: Caitie and Tuan

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

### Exercise 1: Construct Dictionaries from Data

<img src="lab4_exercise1.png" width="1000">

In [1]:
import pathlib  # Use pathlib to work with paths
def convert_csv_to_dict(file_path):
    # Create an object of file path
    a_path=pathlib.Path(file_path)
    # Extract file name from file path
    file_name=a_path.name
    # Load csv file using read_csv()
    csv_object = pd.read_csv(file_name)
    # Create a dictionary object
    dict_object={}
    # Set the first key and its value for the dictionary
    dict_object["Filename"] = file_name
    # Convert csv file to Numpy array with .to_numpy() 
    csv_object_np = csv_object.to_numpy()
    # create a variable for the order of the csv object column 
    i=0
    # Implement for loop, traverse all the attributes of csv object
    # The attributes of csv object is the key of the dictionary object
    for name in csv_object.columns:
        # Set the value for each key of the dictionary object
        # The value of each key is a numpy array taking the values in all the rows of i_th column of csv object 
        dict_object[name] = np.array(csv_object_np[:, i])
        # Increase the order of column by 1
        i=i+1
    
    return dict_object

In [ ]:
TSLA_dict = convert_csv_to_dict('your file path to TSLA.csv')

In [ ]:
# Navigate to keys corresponding to 2nd and 4th columns (Open and Low prices) of TSLA.csv, 
# Print first 10 elements of each key.

TSLA_column2 = "Open"
TSLA_column4 = "Low"
print(TSLA_dict[TSLA_column2][:10])
print(TSLA_dict[TSLA_column4][:10])

In [ ]:
diabetes_dict = convert_csv_to_dict('your file path to diabetes.csv')

In [ ]:
# Navigate to keys corresponding to 2nd and 4th columns (Glucose and SkinThickness) of diabetes.csv, 
# Print first 10 elements of each key.

diabetes_column2 = "Glucose"
diabetes_column4 = "SkinThickness"
print(diabetes_dict[diabetes_column2][:10])
print(diabetes_dict[diabetes_column4][:10])

### Exercise 2: Bar graph with confidence intervals

<img src="lab4_exercise2.png" width="1000">

In [ ]:
# Load diabetes.csv 
import scipy.stats as st                # Import scipy.stats to use pre-built statstical functions

diabetes = pd.read_csv('diabetes.csv')
# Convert to Numpy array with .to_numpy() 
diabetes_np = diabetes.to_numpy() 
# Extract rows index with diabetes and no diabetes using Boolean masks
diabetes_pos_ind = diabetes_np[:, -1] == 1  
diabetes_neg_ind = diabetes_np[:, -1] == 0 
# Split dataset into two subsets of diabetic and non-diabetic
diabetes_np_pos = diabetes_np[diabetes_pos_ind, :]  
diabetes_np_neg = diabetes_np[diabetes_neg_ind, :]

In [ ]:
# Extract glucose, blood pressure, and BMI metrics from diabetic and non-diabetic

# Extract glucose from non-diabetic metrics
non_diabetic_glucose = diabetes_np_neg[:,1]
# Extract blood pressure from non-diabetic metrics
non_diabetic_bp = diabetes_np_neg[:,2]
# Extract BMI from non-diabetic metrics
non_diabetic_bmi = diabetes_np_neg[:,5]

# Extract glucose from diabetic metrics
diabetic_glucose =  diabetes_np_pos[:,1]
# Extract blood pressure from diabetic metrics
diabetic_bp = diabetes_np_pos[:,2]
# Extract BMI from non-diabetic metrics
diabetic_bmi = diabetes_np_pos[:,5]
# Create a list of three 1D arrays each corresponding to glucose, blood pressure, and BMI 
non_diabetic_list = [non_diabetic_glucose, non_diabetic_bp, non_diabetic_bmi]
diabetic_list = [diabetic_glucose, diabetic_bp, diabetic_bmi]
# Create a list of three bar labels 
non_diabetic_bar_labels = ['non_diabetic_glucose', 'non_diabetic_bp', 'non_diabetic_bmi']
diabetic_bar_labels = ['diabetic_glucose', 'diabetic_bp', 'diabetic_bmi']

In [ ]:
def produce_bargraph_CI(data_vec_list, conf_level, bar_labels):
    # Find the length of the data list
    x = len(data_vec_list)
    # Create original arrays with the size x to do for loop
    CI_lower = np.zeros(x)
    CI_upper = np.zeros(x)
    h = np.zeros(x)
    # Implement for loop, find the lower and upper bound for provided confidence level
    for i in range(0,x):
        CI_lower[i], CI_upper[i] = st.t.interval(alpha=conf_level, df=len(data_vec_list[i])-1, 
                                      loc=np.mean(data_vec_list[i]), scale=st.sem(data_vec_list[i]))
        # st.t.interval() computes lower and upper bound for provided confidence level
        # alpha - confidence level
        # df - degree of freedom (size of the data - 1)
        # loc - The mean value of the data
        # scale = standard error of the data
        h[i] = CI_upper[i] - np.mean(data_vec_list[i])        # Confidence interval size
        plt.bar([bar_labels[i]], [np.mean(data_vec_list[i])], # Define x and y-axis data for the bar
        width = 0.4, color = 'black',                         # Define visual property of the main bar
        yerr = [h[i]], ecolor = 'grey',                       # Define the confidence interval size and error bar color
        error_kw=dict(lw=5, capsize=30, capthick=1))          # Define visual properties of the confidence interval

In [ ]:
produce_bargraph_CI(data_vec_list = non_diabetic_list, conf_level = 0.99, bar_labels = non_diabetic_bar_labels)

In [ ]:
produce_bargraph_CI(data_vec_list = diabetic_list, conf_level = 0.95, bar_labels = diabetic_bar_labels)

### Exercise 3: Rolling Mean/Median Function from Scratch

<img src="lab4_exercise3.png" width="1000">

In [ ]:
# Load stock datasets

tesla = pd.read_csv('TSLA.csv') 
tesla_np = tesla.to_numpy()

google = pd.read_csv('GOOGL.csv') 
google_np = google.to_numpy()

dji = pd.read_csv('DJI.csv') 
dji_np = dji.to_numpy()

In [ ]:
# Extract closing price for each stock data

tesla_np_closing = tesla_np[:, 4]
google_np_closing = google_np[:, 4]
DJI_np_closing = dji_np[:, 4]

In [ ]:
def smooth_data(data_arr, smooth_type, window_size):
    # Find the length of the data array
    x= len(data_arr)
    # Set the length of the smoothed data array, rolling operation reduces the data length by (window size - 1)
    smoothed_data_arr = np.zeros(x-(window_size-1))
    # Implement for loop x-(window_size-1) times
    for i in range(x-(window_size-1)):
         # Implement conditional statements for 2 cases of smoothing
        if smooth_type=='mean':
            smoothed_data_arr[i] = np.ceil(np.mean(data_arr[i:window_size-1+i]))
        else: #smooth_type == 'median'
            smoothed_data_arr[i] = np.median(data_arr[i:window_size-1+i]) 
    # Return smoothed data array
    return smoothed_data_arr

In [ ]:
# Tesla closing prices, smooth_type = 'mean', window_size = 100
# Note your smoothed data will be shorter than the original

smoothed_tsla_closing  = smooth_data(tesla_np_closing, smooth_type = 'mean', window_size = 100)

In [ ]:
# plot smoothed_tsla_closing on top of tesla_np_closing
plt.subplot(311)
# Plot smoothed tesla closing prices with linestyle='dashed'
plt.plot(smoothed_tsla_closing, linewidth = 2, color = 'red',linestyle='dashed' ) 
# Plot original tesla closing prices with linestyle='solid'
plt.plot(tesla_np_closing, linewidth = 2, color = 'black',linestyle='solid')


In [ ]:
# Google closing prices, smooth_type = 'median', window_size = 150
# Note your smoothed data will be shorter than the original

smoothed_google_closing  = smooth_data(google_np_closing, smooth_type = 'median', window_size = 150)

In [ ]:
# plot smoothed_google_closing on top of google_np_closing

plt.subplot(312)
# Plot smoothed google closing prices with linestyle='dashed'
plt.plot(smoothed_google_closing, linewidth = 2, color = 'red', linestyle='dashed') 
# Plot original google closing prices with linestyle='solid'
plt.plot(google_np_closing, linewidth = 2, color = 'black', linestyle='solid')

In [ ]:
# Dow Jones Index closing prices, smooth_type = 'mean', window_size = 200
# Note your smoothed data will be shorter than the original

smoothed_dji_closing  = smooth_data(DJI_np_closing, smooth_type = 'mean', window_size = 200)

In [ ]:
# plot smoothed_dji_closing on top of dji_np_closing

plt.subplot(313)
# Plot smoothed DJI closing prices with linestyle='dashed'
plt.plot(smoothed_dji_closing, linewidth = 1, color = 'red', linestyle='dashed' ) 
# Plot original DJI closing prices with linestyle='solid'
plt.plot(DJI_np_closing, linewidth = 1, color = 'black', linestyle='solid')

## Extra credit: Code efficiency
### Achieve a runtime speed of < 50ms

In [ ]:
timeit -n 1 -r 7 smoothed_google_closing  = smooth_data(google_np_closing, smooth_type = 'median', window_size = 150)

### Exercise 4: Ranking Daily Stock Surges/Crashes

<img src="lab4_exercise4.png" width="1000">

In [ ]:
def detect_surge_crash(filepath, detect_type, num_output_dates):
    
    # Create an object of file path
    a_path=pathlib.Path(filepath)
    # Extract file name from file path
    filename=a_path.name
    # Load csv file using read_csv()
    csv_object = pd.read_csv(filename)
    # Convert csv file to Numpy array with .to_numpy() 
    csv_object_np = csv_object.to_numpy()
    # Extract close data, open data and dates from csv_object
    close_data=csv_object_np[:,4]
    open_data=csv_object_np[:,1]
    date_data=csv_object_np[:,0]
    # Find the length of data array
    x=len(close_data)
    # Create an original list with size x to store the data of price changes.
    price_change=list(range(x))
    # Create empty lists of surge and crash data, surge and crash dates because of their unknown size
    surge_data=[]
    crash_data=[]
    dates_surge=[]
    dates_crash=[]
    # Implement for loop x times to calculate the price changes, split into two cases of surge and crash.
    for i in range(0,x):
        # price change=close_data-open_data
        price_change[i]=close_data[i]-open_data[i]
        # Implement conditional statements to split price changes and dates into two cases of surge and crash.
        if price_change[i]>0: # If price change is positive, append it to the surge data list 
                              # and the corresponding dates is appended to surge dates list            
            surge_data.append(price_change[i])
            dates_surge.append(date_data[i])
        else:  # If price change is negative, append it to the crash data list 
               # and the corresponding dates is appended to crash dates list
            crash_data.append(price_change[i])
            dates_crash.append(date_data[i])
    # Sorting the indices of surge and crash lists that their values are from low to high
    sort_indices_surge=np.argsort(surge_data)
    sort_indices_crash=np.argsort(np.abs(crash_data)) # Take absolute values because the crash data is negative
    # Find the length of surge and crash lists
    y=len(surge_data)
    # Create original surge and crash lists with the size y to implement for loop 
    surge_data_sorted=list(range(y))
    crash_data_sorted=list(range(y))
    dates_surge_sorted=list(range(y))
    dates_crash_sorted=list(range(y))
    # Implement for loop to sort the values of surge and crash lists from largest to smallest
    for j in range(0,y):
        # The indices are sorted from low to high following their values, so the largest data is taken from the last index.
        surge_data_sorted[j]=surge_data[sort_indices_surge[y-1-j]]
        dates_surge_sorted[j]= dates_surge[sort_indices_surge[y-1-j]]
        crash_data_sorted[j]=np.abs(crash_data[sort_indices_crash[y-1-j]])
        dates_crash_sorted[j]= dates_crash[sort_indices_surge[y-1-j]]
    # Implement conditional statements for two cases of surge and crash,
    # and limit the length of the output lists to  num_output_dates
    if detect_type=='surge':
        date_list=dates_surge_sorted[:num_output_dates]
        price_change_list=surge_data_sorted[:num_output_dates]
    else: # detect_type=='crash'
        date_list=dates_crash_sorted[:num_output_dates]
        price_change_list=crash_data_sorted[:num_output_dates]
    
    return date_list, price_change_list

In [ ]:
date_list_t, price_change_list_t = detect_surge_crash(filepath = path to TSLA.csv, detect_type = 'surge', num_output_dates = 5)

In [ ]:
print(date_list_t, price_change_list_t)

In [ ]:
date_list_g, price_change_list_g = detect_surge_crash(filepath = path to GOOGL.csv, detect_type = 'crash', num_output_dates = 7)

In [ ]:
print(date_list_g, price_change_list_g)

## Extra credit: Code efficiency
### Achieve a runtime speed of < 10ms

In [ ]:
timeit -n 1 -r 7 date_list_t, price_change_list_t = detect_surge_crash(filepath = path to TSLA.csv, detect_type = 'surge', num_output_dates = 5)

### Exercise 5: Human Debugger

<img src="lab4_exercise5.png" width="1200">

### Faulty function #1 

In [ ]:
def average_data_per_col(arr_2d):
    
    # NOTE FROM YOUR FRIEND PREPARING FOR STARBUCKS SOFTWARE ENGINEER TECH INTERVIEW 
    """  The function takes numpy 2d array as an input, computes mean for each column data, and outputs 1D array 
         with the length equal to the # of columns.
         
         For some reason I keep getting errors.... I need your help to debug the code.
         I need this position so I can get free ice lattes... :(
    
    """
    
    # placeholder for averaged values
    averaged_data = ()
    
    # Loop through each column data to compute mean and append to averaged_data 
    for k in range(len(arr_2d[:, 0])):
        
        averaged_column_data = np.mean(arr_2d[:, k])
        averaged_data.append(averaged_column_data)
        
    # Return numpy array form of the averaged_data
    return np.array(averaged_data)

In [ ]:
# Load diabetes.csv and convert to numpy array

diabetes = pd.read_csv('diabetes.csv')
diabetes_np = diabetes.to_numpy()

In [ ]:
# Run faulty function 1

averaged_diabetic_attributes =  average_data_per_col(diabetes_np)

In [ ]:
# YOUR FIXED FUNCTION HERE
# Make sure you comment on your fixes

def average_data_per_col_fixed(arr_2d):
    
    # placeholder for averaged values
    averaged_data = []  # {} -> []
                        # It should be a list to append an element, 'tuple' object has no attribute 'append'
    
    # Loop through each column data to compute mean and append to averaged_data 
    for k in range(len(arr_2d[0, :])):
        # len(arr_2d[:, 0]) -> len(arr_2d[0, :])
        # k refers to the number of columns of arr_2d because we need find average value of each column,
        # so we need to find the length of the first row in arr_2d, not the length of the first column.
        averaged_column_data = np.mean(arr_2d[:, k])
        averaged_data.append(averaged_column_data)
        
    # Return numpy array form of the averaged_data
    return np.array(averaged_data)
        

In [ ]:
# Test your fixed function

averaged_diabetic_attributes =  average_data_per_col_fixed(diabetes_np)

In [ ]:
# Compare with correct results

correct_result_func1 = np.load('E5_correct_output_1.npy')

# Should return True if the result is correct
np.sum(np.round(correct_result_func1, 3) == np.round(averaged_diabetic_attributes, 3)) == len(correct_result_func1) 

### Faulty function #2

In [ ]:
def daily_stock_change_2_normalized_percentage(opening_price_arr, closing_price_arr):
    
    # NOTE FROM YOUR FRIEND WHO INVESTED IN TESLA
    """  I want to write a function which takes 2 1D numpy arrays of each corresponding to opening/closing prices of stock
         and output 1D array of daily stock change in percentages. 
         
         I want the percentages scale to be in a scale such that 1 = 100%, -0.5 = -50%, 1.5 = 150%  etc.
         For example, day 1 opening: $10, day 1 closing: $15, change in scaled percecntage: 0.5.
         
         I am not really getting errors but the numbers don't look right... Can you help me?? :'( 
    
    """
    
    # placeholder for change percentage values
    change_percentages = np.zeros(len(opening_price_arr), dtype = 'int')
    
    # Loop through each opening/closing price to compute the change percentage
    for date_num in range(len(opening_price_arr)):
        
        change_percentages[date_num] = opening_price_arr[date_num] - closing_price_arr[date_num] / opening_price_arr[date_num]
    
    return change_percentages

In [ ]:
# Load tsla.csv and convert to numpy array

tesla = pd.read_csv('TSLA.csv') 
tesla_np = tesla.to_numpy()

In [ ]:
# Run faulty function 2

change_percentages = daily_stock_change_2_normalized_percentage(tesla_np[:, 1], tesla_np[:, 4])
print(change_percentages)

In [1]:
# YOUR FIXED FUNCTION HERE
# Make sure you comment on your fixes

def daily_stock_change_2_normalized_percentage_fixed(opening_price_arr, closing_price_arr):
    
    # placeholder for change percentage values
    # Because the percentages scale is not integer, we should not set dtype='int' for the value of change_percentages array.
    change_percentages = np.zeros(len(opening_price_arr))
    
    # Loop through each opening/closing price to compute the change percentage
    for date_num in range(len(opening_price_arr)):
        # Because division takes precedence over subtraction, we should enclose the subtraction in parentheses.
        change_percentages[date_num] = (closing_price_arr[date_num]-opening_price_arr[date_num]) / opening_price_arr[date_num]
    
    return change_percentages
    

In [ ]:
# Test your fixed function

change_percentages = daily_stock_change_2_normalized_percentage_fixed(tesla_np[:, 1], tesla_np[:, 4])
print(change_percentages)

In [ ]:
# Compare with correct results

correct_result_func2 = np.load('E5_correct_output_2.npy')

# Should return True if the result is correct
np.sum(np.round(correct_result_func2, 3) == np.round(change_percentages, 3)) == len(correct_result_func2) 

### Faulty function #3

In [ ]:
def subset_diabetes_by_age(diabetes_data):
    
    # NOTE FROM YOUR FRIEND WHO WORKS AT UW IHME
    """ The function takes diabetes pandas data frame as an input and outputs a subplot of 3 x 1 with three histograms.    
    
        Specifically, I want to divide the diabetes data into three age groups - 
            1. 20 to 40
            2. 40 to 60
            3. 60 to 80
            
        and plot 3 histograms of glucose distribution (50 bins per histogram) in 3 x 1 python subplots.
        
        I seem to be getting error from very beginning even before I divide the dataset....
        My coworkers at IHME prefer R rather than Python... so you are my only hope!  
    
    """
    
    # Extract the age column of the diabetes_data 
    age_column = diabetes_data[:, 8] 
    
    # Construct boolean mask for each age group 
    age_20_40_bool_mask = age_column > 20 + age_column < 40
    age_40_60_bool_mask = age_column > 40 + age_column < 60
    age_60_80_bool_mask = age_column > 60 + age_column < 80
    
    # Get glucose data for each age group via applying the boolean mask for each age group
    age_20_40_glucose = diabetes_data[age_20_40_bool_mask, 2]
    age_40_60_glucose = diabetes_data[age_40_60_bool_mask, 2]
    age_60_80_glucose = diabetes_data[age_60_80_bool_mask, 2]
    
    # Plot the histogram for each age group in 3 x 1 subplot
    fig = plt.figure(figsize=(15,7))
    
    plt.subplot(3,1,1)
    
    plt.hist(age_20_40_glucose, bins = 50)
    plt.title('Age 20 to 40', fontsize = 15)
    
    plt.subplot(3,1,2)
    
    plt.hist(age_40_60_glucose, bins = 50)
    plt.title('Age 40 to 60', fontsize = 15)
    
    plt.subplot(3,1,3)
    
    plt.hist(age_60_80_glucose, bins = 50)
    plt.title('Age 60 to 80', fontsize = 15)
    
    plt.tight_layout()
    
    plt.show()

In [ ]:
# Load diabetes.csv as pandas dataframe

diabetes = pd.read_csv('diabetes.csv')

In [ ]:
# Run faulty function 3

subset_diabetes_by_age(diabetes_data = diabetes)

In [ ]:
# YOUR FIXED FUNCTION HERE
# Make sure you comment on your fixes

def subset_diabetes_by_age_fixed(diabetes_data):
    # Because the parameter diabetes_data is given in csv file, we firstly should convert it to Numpy array 
    diabetes_np = diabetes_data.to_numpy()
    # Extract the age column of the diabetes_data 
    age_column = diabetes_np[:, 7] # The column represents for the age should be 7 , not 8
    
    # Construct boolean mask for each age group 
    # The '+' operation gives True for all the boolean outputs, it should be change into '&'
    # Comparison operation should be enclosed by parentheses 
    # to eliminate the error "the truth value of an array with more than one element is ambiguous"
    age_20_40_bool_mask= (age_column > 20) & (age_column < 40)
    age_40_60_bool_mask= (age_column > 40) & (age_column < 60)
    age_60_80_bool_mask= (age_column > 60) & (age_column < 80)
    
    # Get glucose data for each age group via applying the boolean mask for each age group
    # The column represents for the glucose should be 1 , not 2
    age_20_40_glucose = diabetes_np[age_20_40_bool_mask, 1]
    age_40_60_glucose = diabetes_np[age_40_60_bool_mask, 1]
    age_60_80_glucose = diabetes_np[age_60_80_bool_mask, 1]
    
    # Plot the histogram for each age group in 3 x 1 subplot
    fig = plt.figure(figsize=(15,7))
    
    plt.subplot(3,1,1)
    
    plt.hist(age_20_40_glucose,color='blue', bins = 50)
    plt.title('Age 20 to 40', fontsize = 15)
    
    plt.subplot(3,1,2)
    
    plt.hist(age_40_60_glucose,color='blue', bins = 50)
    plt.title('Age 40 to 60', fontsize = 15)
    
    plt.subplot(3,1,3)
    
    plt.hist(age_60_80_glucose,color='blue', bins = 50)
    plt.title('Age 60 to 80', fontsize = 15)
    
    plt.tight_layout()
    
    plt.show()
    

In [ ]:
# Test your fixed function
# Compare your plot with the correct plot provided in template folder

subset_diabetes_by_age_fixed(diabetes_data = diabetes)